In [1]:
import requests
from bs4 import BeautifulSoup
from newspaper import Article  
import csv 
import pandas as pd
import numpy as np

In [2]:
url="https://in.mashable.com/"
r=requests.get(url)

In [3]:
soup=BeautifulSoup(r.content,'html5lib') 
table=soup.findAll('a',attrs={'class':'title'})

In [4]:
news=[]
for row in table: 
    if not row['href'].startswith('http'):
        news.append('https://in.mashable.com'+row['href'])

In [5]:
df=[]
for i in news:
    article = Article(i, language="en")
    article.download() 
    article.parse() 
    article.nlp() 
    data={}
    data['Title']=article.title
    data['Text']=article.text
    data['Summary']=article.summary
    data['Keywords']=article.keywords
    df.append(data)

In [6]:
dataset=pd.DataFrame(df)
dataset.head()

,Keywords,Summary,Text,Title
0,"[ftc, tool, facebooks, agreement, peoples, app...",Facebook announced last week that a federal co...,Facebook announced last week that a federal co...,U.S. Court Approves Facebook’s $5 Billion Priv...
1,"[learning, improve, service, develop, stanford...",This streaming service used the Fugu algorithm...,At a time when millions are stuck at home and ...,Stanford Researchers Develop Algorithm To Impr...
2,"[charger, india, support, pro, leak, suggests,...",OnePlus recently launched the OnePlus 8 and On...,OnePlus recently launched the OnePlus 8 and On...,OnePlus Warp Charge 30 Wireless Charger Leak S...
3,"[north, wonsan, health, jongun, rumours, train...","Over the past week, North Korean leader Kim Jo...","Over the past week, North Korean leader Kim Jo...",Is Kim Jong-un Alive? Here’s What We Know Abou...
4,"[director, movie, marvel, sam, scenes, action,...",Firstly because it was Chris Hemsworth's first...,The buildup to Extraction was quite thrilling....,Exclusive: 'Extraction' Director Sam Hargrave ...


In [7]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

In [8]:
FILEPATH="OnlineNewsPopularity.csv"

In [9]:
data1=pd.read_csv(FILEPATH)
data1.head(5)

,url,timedelta,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,...,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,shares
0,http://mashable.com/2013/01/07/amazon-instant-...,731.0,12.0,219.0,0.663594,1.0,0.815385,4.0,2.0,1.0,...,0.100000,0.7,-0.350000,-0.600,-0.200000,0.500000,-0.187500,0.000000,0.187500,593
1,http://mashable.com/2013/01/07/ap-samsung-spon...,731.0,9.0,255.0,0.604743,1.0,0.791946,3.0,1.0,1.0,...,0.033333,0.7,-0.118750,-0.125,-0.100000,0.000000,0.000000,0.500000,0.000000,711
2,http://mashable.com/2013/01/07/apple-40-billio...,731.0,9.0,211.0,0.575130,1.0,0.663866,3.0,1.0,1.0,...,0.100000,1.0,-0.466667,-0.800,-0.133333,0.000000,0.000000,0.500000,0.000000,1500
3,http://mashable.com/2013/01/07/astronaut-notre...,731.0,9.0,531.0,0.503788,1.0,0.665635,9.0,0.0,1.0,...,0.136364,0.8,-0.369697,-0.600,-0.166667,0.000000,0.000000,0.500000,0.000000,1200
4,http://mashable.com/2013/01/07/att-u-verse-apps/,731.0,13.0,1072.0,0.415646,1.0,0.540890,19.0,19.0,20.0,...,0.033333,1.0,-0.220192,-0.500,-0.050000,0.454545,0.136364,0.045455,0.136364,505


In [10]:
clean_cols={x: x.lower().strip() for x in list(data1)}
data1=data1.rename(index=str,columns=clean_cols)

In [11]:
train_set, test_set = train_test_split(data1, test_size=0.4, random_state=42)

x_train = train_set.drop(['url','shares', 'timedelta','num_self_hrefs', 'kw_min_min', 'kw_max_min', 'kw_avg_min','kw_min_max','kw_max_max','kw_avg_max','kw_min_avg','kw_max_avg','kw_avg_avg','self_reference_min_shares','self_reference_max_shares','self_reference_avg_sharess'], axis=1)
y_train = train_set['shares']

x_test = test_set.drop(['url','shares', 'timedelta','num_self_hrefs', 'kw_min_min', 'kw_max_min', 'kw_avg_min','kw_min_max','kw_max_max','kw_avg_max','kw_min_avg','kw_max_avg','kw_avg_avg','self_reference_min_shares','self_reference_max_shares','self_reference_avg_sharess'], axis=1)
y_test = test_set['shares']

In [12]:
clf=RandomForestRegressor(random_state=42)
clf.fit(x_train,y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=42, verbose=0, warm_start=False)

In [13]:
result=pd.DataFrame(clf.predict(x_train),list(y_train))

In [14]:
result.reset_index(level=0, inplace=True)
result_df = result.rename(index=str, columns={"index": "Actual shares", 0: "Predicted shares"})
result_df.head()

,Actual shares,Predicted shares
0,1900,2016.82
1,1800,1777.14
2,790,922.30
3,872,1273.54
4,2500,2629.21


In [15]:
import pandas as pd
data = pd.read_csv('abcnews-date-text.csv', error_bad_lines=False);
data_text = data[['headline_text']]
data_text['index'] = data_text.index
documents = data_text
print(len(documents))
print(documents[:5])

1186018
                                       headline_text  index
0  aba decides against community broadcasting lic...      0
1     act fire witnesses must be aware of defamation      1
2     a g calls for infrastructure protection summit      2
3           air nz staff in aust strike for pay rise      3
4      air nz strike to affect australian travellers      4


In [16]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk

stemmer=PorterStemmer()

C:\Users\VISHAL\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [17]:
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [18]:
doc_sample = documents[documents['index'] ==2].values[0][0]
print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))

original document: 
['a', 'g', 'calls', 'for', 'infrastructure', 'protection', 'summit']


 tokenized and lemmatized document: 
['call', 'infrastructur', 'protect', 'summit']


In [19]:
processed_docs = documents['headline_text'].map(preprocess)
processed_docs[:10]

0               [decid, commun, broadcast, licenc]
1                               [wit, awar, defam]
2           [call, infrastructur, protect, summit]
3                      [staff, aust, strike, rise]
4             [strike, affect, australian, travel]
5               [ambiti, olsson, win, tripl, jump]
6           [antic, delight, record, break, barca]
7    [aussi, qualifi, stosur, wast, memphi, match]
8            [aust, address, secur, council, iraq]
9                         [australia, lock, timet]
Name: headline_text, dtype: object

In [20]:
dictionary = gensim.corpora.Dictionary(processed_docs)
print(dictionary)
count=0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 15:
        break

Dictionary(67259 unique tokens: ['broadcast', 'commun', 'decid', 'licenc', 'awar']...)
0 broadcast
1 commun
2 decid
3 licenc
4 awar
5 defam
6 wit
7 call
8 infrastructur
9 protect
10 summit
11 aust
12 rise
13 staff
14 strike
15 affect


In [21]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
bow_corpus[2]

[(7, 1), (8, 1), (9, 1), (10, 1)]

In [22]:
bow_doc_2 = bow_corpus[2]
for i in range(len(bow_doc_2)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_2[i][0], 
                                               dictionary[bow_doc_2[i][0]], 
bow_doc_2[i][1]))

Word 7 ("call") appears 1 time.
Word 8 ("infrastructur") appears 1 time.
Word 9 ("protect") appears 1 time.
Word 10 ("summit") appears 1 time.


In [23]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=5, id2word=dictionary)
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.026*"polic" + 0.016*"charg" + 0.014*"court" + 0.012*"death" + 0.012*"murder" + 0.011*"crash" + 0.009*"woman" + 0.008*"face" + 0.008*"kill" + 0.007*"attack"
Topic: 1 
Words: 0.022*"australia" + 0.013*"year" + 0.010*"australian" + 0.008*"open" + 0.006*"win" + 0.006*"coast" + 0.006*"record" + 0.006*"gold" + 0.006*"north" + 0.006*"queensland"
Topic: 2 
Words: 0.012*"trump" + 0.010*"interview" + 0.009*"fund" + 0.009*"rural" + 0.009*"news" + 0.008*"farmer" + 0.008*"say" + 0.008*"health" + 0.007*"chang" + 0.007*"warn"
Topic: 3 
Words: 0.010*"hous" + 0.009*"council" + 0.008*"plan" + 0.008*"elect" + 0.008*"countri" + 0.005*"hour" + 0.005*"power" + 0.005*"govern" + 0.005*"commun" + 0.004*"head"
Topic: 4 
Words: 0.012*"world" + 0.008*"sydney" + 0.008*"water" + 0.007*"time" + 0.006*"say" + 0.006*"green" + 0.005*"island" + 0.005*"royal" + 0.005*"govern" + 0.005*"elect"


In [24]:
for index, score in sorted(lda_model[bow_corpus[2]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 5)))


Score: 0.4493105411529541	 
Topic: 0.012*"trump" + 0.010*"interview" + 0.009*"fund" + 0.009*"rural" + 0.009*"news"

Score: 0.42877814173698425	 
Topic: 0.012*"world" + 0.008*"sydney" + 0.008*"water" + 0.007*"time" + 0.006*"say"

Score: 0.041346970945596695	 
Topic: 0.010*"hous" + 0.009*"council" + 0.008*"plan" + 0.008*"elect" + 0.008*"countri"

Score: 0.04031626507639885	 
Topic: 0.022*"australia" + 0.013*"year" + 0.010*"australian" + 0.008*"open" + 0.006*"win"

Score: 0.040248073637485504	 
Topic: 0.026*"polic" + 0.016*"charg" + 0.014*"court" + 0.012*"death" + 0.012*"murder"


In [25]:
def get_LDA(text):
    unseen_document = text
    bow_vector = dictionary.doc2bow(preprocess(unseen_document))
    result=[]
    for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
        #print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))
        result.append(score)
    return result

In [26]:
from nltk.tokenize import word_tokenize 
from nltk.corpus import stopwords
stopwords=set(stopwords.words('english'))

In [27]:
def for_unique_tokens_rate(words):
    words=tokenize(words)
    no_order = list(set(words))
    unique_tokens_rate=len(no_order)/len(words)
    return unique_tokens_rate

In [28]:
def for_nonstop_words_rate(words):
    words=tokenize(words)
    filtered_sentence = [w for w in words if not w in stopwords]
    nonstop_words_rate=len(filtered_sentence)/len(words)
    return nonstop_words_rate

In [29]:
def for_unique_nonstop_words_rate(words):
    words=tokenize(words)
    filtered_sentence = [w for w in words if not w in stopwords]
    no_order = list(set(filtered_sentence))
    unique_nonstop_words_rate=len(no_order)/len(words)
    return unique_nonstop_words_rate

In [30]:
def avg_token_length(words):
    words=tokenize(words)
    length=[]
    for i in words:
        length.append(len(i))
    return np.average(length)

In [31]:
from textblob import TextBlob
import datefinder
import datetime  
from datetime import date 

In [32]:
def day(article_text):
    article=article_text
    if len(list(datefinder.find_dates(article_text)))>0:
        date=str(list(datefinder.find_dates(article_text))[0])
        date=date.split()
        date=date[0]
        year, month, day = date.split('-')     
        day_name = datetime.date(int(year), int(month), int(day)) 
        return day_name.strftime("%A")
    return "Monday"

In [33]:
def tokenize(text):
    text=text
    return word_tokenize(text)

In [34]:
def non_neutral_tokens(text):
    processed_docs = preprocess(text)
    speech=nltk.pos_tag(processed_docs)
    tags=["NN","NNP","NNS","JJ","JJR","JJS","RB","RBR","RBS","UH"]
    speech_list=[i[0] for i in speech if i[1] in tags]
    return len(speech_list)

In [35]:
positive_words=[]
negative_words=[]
def polar(words):
    all_tokens=tokenize(words)
    for i in all_tokens:
        analysis=TextBlob(i)
        polarity=analysis.sentiment.polarity
        if polarity>0:
            positive_words.append(i)
        if polarity<0:
            negative_words.append(i)
    return positive_words,negative_words

In [36]:
def rates(words):
    words=polar(words)
    positive=words[0]
    negative=words[1]
    global_rate_positive_words=(len(positive)/len(words))/100
    global_rate_negative_words=(len(negative)/len(words))/100
    polar_positive=[]
    polar_negative=[]
    for i in positive:
        analysis=TextBlob(i)
        polar_positive.append(analysis.sentiment.polarity)
        avg_positive_polarity=analysis.sentiment.polarity
    for j in negative:
        analysis2=TextBlob(j)
        polar_negative.append(analysis2.sentiment.polarity)
        avg_negative_polarity=analysis2.sentiment.polarity
    min_positive_polarity=min(polar_positive)
    max_positive_polarity=max(polar_positive)
    min_negative_polarity=min(polar_negative)
    max_negative_polarity=max(polar_negative)
    avg_positive_polarity=np.average(polar_positive)
    avg_negative_polarity=np.average(polar_negative)
    return global_rate_positive_words,global_rate_negative_words,avg_positive_polarity,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity

In [37]:
df1=[]
for i in news:
    pred_info={}
    article = Article(i, language="en") # en for English 
    article.download() 
    article.parse()
    analysis=TextBlob(article.text)
    polarity=analysis.sentiment.polarity
    title_analysis=TextBlob(article.title)
    pred_info['text']=article.text
    pred_info['n_tokens_title']=len(tokenize(article.title))
    pred_info['n_tokens_content']=len(tokenize(article.text))
    pred_info['n_unique_tokens']=for_unique_tokens_rate(article.text)
    pred_info['n_non_stop_words']=for_nonstop_words_rate(article.text)
    pred_info['n_non_stop_unique_tokens']=for_unique_nonstop_words_rate(article.text)
    pred_info['num_hrefs']=article.html.count("https://in.mashable.com")
    pred_info['num_imgs']=len(article.images)
    pred_info['num_videos']=len(article.movies)
    pred_info['average_token_length']=avg_token_length(article.text)
    pred_info['num_keywords']=len(article.keywords)
    
    if "life-style" in article.url:
        pred_info['data_channel_is_lifestyle']=1
    else:
        pred_info['data_channel_is_lifestyle']=0
    if "etimes" in article.url:
        pred_info['data_channel_is_entertainment']=1
    else:
        pred_info['data_channel_is_entertainment']=0
    if "business" in article.url:
        pred_info['data_channel_is_bus']=1
    else:
        pred_info['data_channel_is_bus']=0
    if "social media" or "facebook" or "whatsapp" in article.text.lower():
        data_channel_is_socmed=1
        data_channel_is_tech=0
        data_channel_is_world=0
    else:
        data_channel_is_socmed=0
    if ("technology" or "tech" in article.text.lower()) or ("technology" or "tech" in article.url):
        data_channel_is_tech=1
        data_channel_is_socmed=0
        data_channel_is_world=0
    else:
        data_channel_is_tech=0
    if "world" in article.url:
        data_channel_is_world=1
        data_channel_is_tech=0
        data_channel_is_socmed=0
    else:
        data_channel_is_world=0
        
    pred_info['data_channel_is_socmed']=data_channel_is_socmed
    pred_info['data_channel_is_tech']=data_channel_is_tech
    pred_info['data_channel_is_world']=data_channel_is_world
    
    if day(i)=="Monday":
        pred_info['weekday_is_monday']=1
    else:
        pred_info['weekday_is_monday']=0
    if day(i)=="Tuesday":
        pred_info['weekday_is_tuesday']=1
    else:
        pred_info['weekday_is_tuesday']=0
    if day(i)=="Wednesday":
        pred_info['weekday_is_wednesday']=1
    else:
        pred_info['weekday_is_wednesday']=0
    if day(i)=="Thursday":
        pred_info['weekday_is_thursday']=1
    else:
        pred_info['weekday_is_thursday']=0
    if day(i)=="Friday":
        pred_info['weekday_is_friday']=1
    else:
        pred_info['weekday_is_friday']=0
    if day(i)=="Saturday":
        pred_info['weekday_is_saturday']=1
        pred_info['is_weekend']=1
    else:
        pred_info['weekday_is_saturday']=0
    if day(i)=="Sunday":
        pred_info['weekday_is_sunday']=1
        pred_info['is_weekend']=1
    else:
        pred_info['weekday_is_sunday']=0
        pred_info['is_weekend']=0
        
    pred_info['LDA_00']=get_LDA(article.text)[0]
    pred_info['LDA_01']=get_LDA(article.text)[1]
    pred_info['LDA_02']=get_LDA(article.text)[2]
    pred_info['LDA_03']=get_LDA(article.text)[3]
    pred_info['LDA_04']=get_LDA(article.text)[4]
    pred_info['global_subjectivity']=analysis.sentiment.subjectivity
    pred_info['global_sentiment_polarity']=analysis.sentiment.polarity
    pred_info['global_rate_positive_words']=rates(article.text)[0]
    pred_info['global_rate_negative_words']=rates(article.text)[1]
    pred_info['avg_positive_polarity']=rates(article.text)[2]
    pred_info['min_positive_polarity']=rates(article.text)[3]
    pred_info['max_positive_polarity']=rates(article.text)[4]
    pred_info['avg_negative_polarity']=rates(article.text)[5]
    pred_info['min_negative_polarity']=rates(article.text)[6]
    pred_info['max_negative_polarity']=rates(article.text)[7]
    pred_info['rate_positive_words']=len(polar(article.text)[0])/non_neutral_tokens(article.text)
    pred_info['rate_negative_words']=len(polar(article.text)[1])/non_neutral_tokens(article.text)
    pred_info['title_subjectivity']=title_analysis.sentiment.subjectivity
    pred_info['title_sentiment_polarity']=title_analysis.sentiment.polarity
    pred_info['abs_title_subjectivity']=abs(title_analysis.sentiment.subjectivity-0.5)
    pred_info['abs_title_sentiment_polarity']=abs(title_analysis.sentiment.polarity)
    df1.append(pred_info)

In [38]:
pred_df=pd.DataFrame(df1)
pred_test=pred_df.drop(['text'],axis=1)
pred_df.head()

,LDA_00,LDA_01,LDA_02,LDA_03,LDA_04,abs_title_sentiment_polarity,abs_title_subjectivity,average_token_length,avg_negative_polarity,avg_positive_polarity,...,text,title_sentiment_polarity,title_subjectivity,weekday_is_friday,weekday_is_monday,weekday_is_saturday,weekday_is_sunday,weekday_is_thursday,weekday_is_tuesday,weekday_is_wednesday
0,0.359953,0.334193,0.112306,0.109795,0.083752,0.000000,0.500000,4.584656,-0.133333,0.309861,...,Facebook announced last week that a federal co...,0.000000,0.000000,0,0,0,1,0,0,0
1,0.252359,0.217821,0.186469,0.177869,0.165486,0.000000,0.500000,4.652695,-0.198889,0.289338,...,At a time when millions are stuck at home and ...,0.000000,0.000000,0,1,0,0,0,0,0
2,0.270169,0.233210,0.199938,0.193831,0.102847,0.000000,0.500000,4.283976,-0.229518,0.276925,...,OnePlus recently launched the OnePlus 8 and On...,0.000000,0.000000,0,0,0,0,1,0,0
3,0.309274,0.258175,0.228486,0.111496,0.092567,0.100000,0.100000,4.370874,-0.227222,0.279629,...,"Over the past week, North Korean leader Kim Jo...",0.100000,0.400000,0,1,0,0,0,0,0
4,0.380239,0.225929,0.146265,0.137576,0.109989,0.027778,0.305556,4.396396,-0.227804,0.277646,...,The buildup to Extraction was quite thrilling....,-0.027778,0.194444,0,1,0,0,0,0,0


In [39]:
test2=pd.DataFrame(clf.predict(pred_test),pred_df['text'])
test2.reset_index(level=0, inplace=True)
test2 = test2.rename(index=str, columns={"index": "News", 0: "Virality"})
test2

,text,Virality
0,Facebook announced last week that a federal co...,63566.25
1,At a time when millions are stuck at home and ...,72694.64
2,OnePlus recently launched the OnePlus 8 and On...,66897.04
3,"Over the past week, North Korean leader Kim Jo...",75233.65
4,The buildup to Extraction was quite thrilling....,75452.64
5,Here's an odd fact: Apple Music is now availab...,75111.04
6,Bill Gates has already pledged $250 million to...,72636.24
7,A newly discovered bug will crash your Apple i...,75233.65
8,Maybe it's that we've all been in quarantine f...,74786.64
9,Being away from the people we love is hard. Pa...,79128.51
